<a href="https://colab.research.google.com/github/deepavjs/SEAR/blob/main/SearML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Kaggle and download the datasets here, unzip and store in directory
#! pip install kaggle
#! mkdir ~/.kaggle
#!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

#! kaggle  datasets download mkechinov/ecommerce-purchase-history-from-electronics-store
#!unzip /content/ecommerce-purchase-history-from-electronics-store.zip -d /content/input

In [ ]:
# Mount Google drive and import data from the training/test directory respectively
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# loop over the list of csv files
def loadCSVFiles(csv_files):
  file_dict = {}
  for f in csv_files:
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("/")[-1])
    key = f.split("/")[-1]
    df = pd.read_csv(f)
    #checkEmptyColumns(df)
    file_dict[key] = df
  return file_dict

In [ ]:
# Load the CSV files from the google drive path
import pandas as pd
import os
import glob
#import pandas_profiling as pp
  
# use glob to get all the csv files 
# from the training folder
trainPath = '/content/drive/MyDrive/training'
print(trainPath)
csv_files = glob.glob(os.path.join(trainPath, "*.csv"))
print(csv_files)
  
files_train_dict={}
files_train_dict = loadCSVFiles(csv_files)

# use glob to get all the csv files 
# from the test folder
testPath = '/content/drive/MyDrive/test'
print(testPath)
csv_test_files = glob.glob(os.path.join(testPath, "*.csv"))
print(csv_test_files)
  
files_test_dict={}
files_test_dict = loadCSVFiles(csv_test_files)

/content/drive/MyDrive/training
['/content/drive/MyDrive/training/2.csv', '/content/drive/MyDrive/training/3.csv', '/content/drive/MyDrive/training/1.csv', '/content/drive/MyDrive/training/4.csv', '/content/drive/MyDrive/training/animals_6.csv', '/content/drive/MyDrive/training/historical_7.csv', '/content/drive/MyDrive/training/historical_8.csv', '/content/drive/MyDrive/training/school_10.csv', '/content/drive/MyDrive/training/hospital_11.csv', '/content/drive/MyDrive/training/health_12.csv', '/content/drive/MyDrive/training/health_13.csv', '/content/drive/MyDrive/training/medical_14.csv', '/content/drive/MyDrive/training/birthrate_15.csv', '/content/drive/MyDrive/training/health_16.csv', '/content/drive/MyDrive/training/revenue_17.csv', '/content/drive/MyDrive/training/medicare_18.csv', '/content/drive/MyDrive/training/population_19.csv', '/content/drive/MyDrive/training/library_20.csv', '/content/drive/MyDrive/training/location_21.csv', '/content/drive/MyDrive/training/location_22.c

In [ ]:
# Build field name list for all the datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import json
import numpy as np
import string
import csv

featureDict = {}
columnDict = {}
wordFrequencyMap = {}
nameClassificationPair = {}
stdDict = {}

def readStdLabel():
  stdLabelPath = '/content/drive/MyDrive/config/std-label.csv'
  std_label_df = pd.read_csv(stdLabelPath)
  return std_label_df

def getStdLabelForColumnName(colName):
  for i in stdDict:
    if i['field'] == colName:
      return i['category']
  return ''
#print(json.dumps(std_dict,default=str, indent=4))

def writeToCSVFile(wordFrequencyMap, toFileName):
  for w in sorted(wordFrequencyMap, key=wordFrequencyMap.get, reverse=True):
    print(w, wordFrequencyMap[w])

  #test.csv - writes the wordfrequency map to this file
  #nameClassificationPairOutput.csv - writes the name and its classification pair to this file
  #For ex: "Arizona" to "State" where Arizona is the name and State is its classification
  with open(toFileName, 'w') as f:
    for key in wordFrequencyMap.keys():
      #print(key)
      f.write("%s,%s\n"%(key,wordFrequencyMap[key]))

def checkEmptyColumns(df):
  #Check if there is any empty columns
  df = df.replace(' ', np.nan)
  nan_cols = []
  for col in df.columns:
    x = pd.isna(df[col])
    x = x.to_numpy()
    if not False in x:
        nan_cols.append(col)
  return nan_cols

def transformColumnName(colName):
  newStr = ""
  newStr = ''.join(e for e in colName if e.isalnum())
  newStr = newStr.lower()
  return newStr

def fieldNameListBuilder(tableList):
  fieldNameList = []
  for tables in tableList:
    tableName = tables["tableName"]
    singleTableList = []
    for fields in tables["fields"]:
      singleTableList.append(tableName + "|" + fields["fieldName"])
    fieldNameList.append(singleTableList)

  return fieldNameList

def buildFeature(records, tableName, num_records):
  #print(records.columns)
  columnDict = {}
  totl = records.describe(include='all')
  #print('totl')
  #print(totl)
  emptyColNames = []
  emptyColNames = checkEmptyColumns(records) 

  for col in records.columns:
    if col in emptyColNames:
      continue
    featureDict = {}
    Count_data = 0
    total_str_len = 0
    total_spl_char_len = 0
    total_chars = 0
    numbers_perc=0
    letters_perc=0
    spaces_perc=0
    symbols_perc=0
    number_of_chars = 0
    colTransformed = transformColumnName(str(col))
    if colTransformed in wordFrequencyMap:
      wordFrequencyMap[colTransformed] = wordFrequencyMap[colTransformed] + 1
    else:
      wordFrequencyMap[colTransformed] = 1
    featureDict["name"] = colTransformed
    stdLabel = getStdLabelForColumnName(colTransformed)
    #print(stdLabel)
    if stdLabel == 'nan':
      featureDict["label"] = 'other'
    else:
      featureDict["label"] = stdLabel
    nameClassificationPair[colTransformed] = stdLabel
    #print(featureDict["label"])
    #print(col)
    #print(totl[col])
    featureDict["max_value_column_content"] = ""
    featureDict["min_value_column_content"] = ""
    featureDict["content_unique_ratio"] = ""
    if totl[col].get('max') != None:
      featureDict["max_value_column_content"] = totl[col].loc['max']
    if totl[col].get('min') != None:
      featureDict["min_value_column_content"] = totl[col].loc['min']
    if totl[col].get('unique') != None:
      featureDict["content_unique_ratio"] = totl[col].loc['unique']
    featureDict["content_histogram"] = ''

    if records[col].dtypes == object:
      na =pd.isna(records[col])
      records[col] = records[col].fillna("")
      strFlag = True
      spl_chars_Flag = True
      for r in records[col]:
        numbers = 0
        #print(tableName)
        #print(records[col])
        #print(r)
        r = str(r)
        if r.isalnum():
          numbers = sum(c.isdigit() for c in r)
          if len(r) > 0:
            numbers_perc = numbers_perc + float(numbers)/float(len(r))
        letters = sum(c.isalpha() for c in r)
        if len(r) > 0:
          letters_perc = letters_perc + float(letters)/float(len(r))
        spaces  = sum(c.isspace() for c in r)
        if len(r) > 0:
          spaces_perc = spaces_perc + float(spaces)/float(len(r))
        symbols  =  len(r) - numbers - letters - spaces
        if len(r) > 0:
          symbols_perc  = symbols_perc + float(len(r) - numbers - letters - spaces)/float(len(r))
        number_of_chars = number_of_chars+numbers+letters+spaces+symbols
      if symbols_perc == 0:
        CountVec = CountVectorizer(lowercase=True,ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                               stop_words='english')
        #transform
        #print(records[col])
        #if len(records[col]) > 0:
        #  print(records[col])
        #  Count_data = CountVec.fit_transform(records[col])

    r1 = records[col].astype(str)
    l = r1.str.len()
    avg = sum(l)/len(l)
    max = r1.str.len().max()
    min = r1.str.len().min()

    #build histogram
    hist = {}
    for i in records[col]:
      hist[i] = hist.get(i, 0) + 1

    featureDict["content_histogram"] = hist
    featureDict["num_unique_unigrams_bow"] = ''#Count_data
    featureDict["avg_number_of_chars"] = number_of_chars/len(l)
    featureDict["percentage_numeric_chars"] = numbers_perc/len(l)
    featureDict["percentage_alphabet_chars"] = letters_perc/len(l)
    featureDict["percentage_space_chars"] = spaces_perc/len(l)
    featureDict["percentage_symbol_chars"] = symbols_perc/len(l)
    featureDict["percentage_num_cells"] = ''
    featureDict["avg_cell_length"] = avg
    featureDict["max_cell_length"] = max
    featureDict["min_cell_length"] = min
    featureDict["possible_bool_field"] = False
    if max <= 5 and max > 0:
      featureDict["possible_bool_field"] = True
    columnDict[col] = featureDict
  #print(wordFrequencyMap)
  #writeToCSVFile(wordFrequencyMap)
  writeToCSVFile(nameClassificationPair,"/content/drive/MyDrive/config/nameClassificationPairOutput.csv")
  return columnDict
  #print(json.dumps(columnDict,default=str, indent=4))
  


In [ ]:
'''import json
stdDict = {}
def getStdLabelForColumnName(colName):
  for i in stdDict:
    if i['field'] == colName:
      return i['category']
  return ''

def readStdLabel():
  stdLabelPath = '/content/drive/MyDrive/config/std-label.csv'
  std_label_df = pd.read_csv(stdLabelPath)
  return std_label_df

df = readStdLabel()
stdDict = df.to_dict('records')
#print(json.dumps(std_dict,default=str, indent=4))
getStdLabelForColumnName('datasource')'''

"import json\nstdDict = {}\ndef getStdLabelForColumnName(colName):\n  for i in stdDict:\n    if i['field'] == colName:\n      return i['category']\n  return ''\n\ndef readStdLabel():\n  stdLabelPath = '/content/drive/MyDrive/config/std-label.csv'\n  std_label_df = pd.read_csv(stdLabelPath)\n  return std_label_df\n\ndf = readStdLabel()\nstdDict = df.to_dict('records')\n#print(json.dumps(std_dict,default=str, indent=4))\ngetStdLabelForColumnName('datasource')"

In [ ]:
# Load the training and test data in memory from the csv files
from tabulate import tabulate
def load_training_test_data(files_dict):
  tableList = []
  for df in files_dict:
    #print(df)
    if len(files_dict[df]) > 1000:
      df_sample = files_dict[df].sample(1000)
    else:
      df_sample = files_dict[df]
    #print(df_sample.columns)
    fileName = df.split(".")[0]
    input_dict = buildFeature(df_sample, fileName, len(df_sample))#1000)
    tableList.append(input_dict)
  return tableList

In [ ]:
## Load training and test data here
tableTrainingList = []
tableTestList = []
std_df = readStdLabel()
stdDict = std_df.to_dict('records')
tableTrainingList = load_training_test_data(files_train_dict)
#print(tableTrainingList)
tableTestList = load_training_test_data(files_test_dict)

Streaming output truncated to the last 5000 lines.
renterfulp100 property
population population
gender pii
agegroup pii
raceorhispanicorigin pii
age pii
sex pii
maritalstatus pii
monthlyrate percentage
percentsalaryhike percentage
program other
federalshare other
federalsharemedicaid other
federalsharearra other
federalsharebipp other
stateshare other
subject other
recordedmessageorrobocall other
allotmentcn other
allotmentstatus other
nepadecapprovedfy other
bioopapprovedfy other
ampapprovedfy other
monitpmteecomplfy other
adminorg other
managingorg other
revdate other
datasource other
grades other
partners other
topics other
methods other
abstract other
providerid other
emergencyservices other
mortalitynationalcomparisonfootnote other
safetyofcarenationalcomparison other
safetyofcarenationalcomparisonfootnote other
readmissionnationalcomparison other
readmissionnationalcomparisonfootnote other
patientexperiencenationalcomparison other
patientexperiencenationalcomparisonfootnote other

In [ ]:
def getTrainingTestDF(tableList):
  dataF = pd.DataFrame()
  itemsDict = {}
  for elem in tableList:
    dataFTemp = pd.DataFrame()
    for rec in elem:
      for item in elem[rec]:
        items=[]
        items.append(elem[rec][item])
        if item in itemsDict.keys():
          temp = itemsDict[item]
          temp.append(elem[rec][item])
          itemsDict[item]=temp
        else:
          itemsDict[item] = items
  dataF = pd.DataFrame(itemsDict) 
  return dataF

With Name as y column, precision score seems to be less
Print the size of ****TRAINING**** data
1260
1260
Print the size of ****TEST**** data
260
260
train score
0.6746031746031746
test score
0.09615384615384616
precision score micro
0.09615384615384616
precision score macro
0.03940613817781448
precision score weighted
0.060581501831501836

In [ ]:

# Import train_test_split function
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

dataTrainingF = pd.DataFrame()
dataTrainingF = getTrainingTestDF(tableTrainingList)

dataTestF = pd.DataFrame()
dataTestF = getTrainingTestDF(tableTestList)
#print(json.dumps(columnDict,default=str, indent=4))
#X=dataF["max_value_column_content"]+ dataF["min_value_column_content"]+dataF["content_unique_ratio"]+dataF["content_histogram"]+dataF["avg_number_of_chars"]+dataF["percentage_numeric_chars"]+dataF["percentage_alphabet_chars"]+dataF["percentage_space_chars"]+dataF["percentage_symbol_chars"]+dataF["percentage_num_cells"]+dataF["avg_cell_length"]+dataF["max_cell_length"]+dataF["min_cell_length"]+dataF["possible_bool_field"]#+dataF["num_unique_unigrams_bow"]
#add content_histogram below, 'percentage_num_cells'
X_train=dataTrainingF[['max_value_column_content', 'min_value_column_content','content_unique_ratio', 'avg_number_of_chars', 'percentage_numeric_chars', 'percentage_alphabet_chars', 'percentage_space_chars', 'percentage_symbol_chars', 'avg_cell_length', 'max_cell_length', 'min_cell_length', 'possible_bool_field']]
y_train=dataTrainingF['label']  # Labels
#print(dataTrainingF['label'])
#Print the size of the data
print("Print the size of ****TRAINING**** data")
print(len(X_train))
print(len(y_train))

X_test=dataTestF[['max_value_column_content', 'min_value_column_content','content_unique_ratio', 'avg_number_of_chars', 'percentage_numeric_chars', 'percentage_alphabet_chars', 'percentage_space_chars', 'percentage_symbol_chars', 'avg_cell_length', 'max_cell_length', 'min_cell_length', 'possible_bool_field']]
y_test=dataTestF['label']  # Labels

print("Print the size of ****TEST**** data")
print(len(X_test))
print(len(y_test))
print("X_test-")
print(X_test)

# Split dataset into training set and test set
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder

from sklearn import metrics
import pylab as pl
# analyze random forest and tune parameters
# grid search
# For the best set of parameters generate the matrix 32x32 and performance metrics
clf = RandomForestClassifier(criterion='gini',
                                 n_estimators=25,
                                 random_state=1,
                                 n_jobs=2)#RandomForestClassifier(max_depth=2, random_state=0)
best_param_dict_rc = {
    'criterion':'gini',
    'n_estimators':25,
    'random_state':1,
    'n_jobs':2,
}
X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
#X_test = X_test.replace(r'^\s*$', np.nan, regex=True)
X_test = X_test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

#print("y_pred")
#print(y_pred)
#print("y_test")
#print(y_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.to_csv('testOutput.csv')

print("X_test")
print(X_test)

#df1 = pd.DataFrame({'y_test': y_test, 'X_test': X_test["name"]})
#df1.to_csv('testOutput1.csv')

# Print scores
train_score = clf.score(X_train,y_train)
print("train score")
print(train_score)

test_score = clf.score(X_test,y_test)
print("test score")
print(test_score)
 
# Print the confusion matrix
#print(metrics.confusion_matrix(y_test, y_pred))
cm = metrics.confusion_matrix(y_test, y_pred)
print(cm)
# Print the precision and recall, among other metrics
#print(metrics.classification_report(y_test, y_pred, digits=3))

from sklearn.metrics import precision_score
print("precision score micro")
prec_score_micro = precision_score(y_test, y_pred, average='micro')
print(prec_score_micro)
print("precision score macro")
prec_score_macro = precision_score(y_test, y_pred, average='macro')
print(prec_score_macro)
print("precision score weighted")
prec_score_wtd=precision_score(y_test, y_pred, average='weighted')
print(prec_score_wtd)



Print the size of ****TRAINING**** data
1260
1260
Print the size of ****TEST**** data
296
296
X_test-
     max_value_column_content  min_value_column_content  content_unique_ratio  \
0                         NaN                       NaN                 998.0   
1                         NaN                       NaN                   2.0   
2                1.504492e+12              1.379203e+12                   NaN   
3                1.504493e+12              1.379203e+12                   NaN   
4                2.220000e+02              2.000000e+00                   NaN   
..                        ...                       ...                   ...   
291                       NaN                       NaN                  30.0   
292                       NaN                       NaN                  45.0   
293              4.157000e+03              1.000000e+00                   NaN   
294              8.450000e+02              1.000000e+00                   NaN   
295    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn import metrics
def regression_results(model):
    score = model.score(X_test, y_test) # this is accuracy
    print(f'The accuracy is {score}')
#    Sensitivity=TP / float(TP+FN)
#    Precision = TP/TP+FP
    

    # obtaining the confusion matrix and making it look nice
    y_pred = model.predict(X_test)
    # must put true before predictions in confusion matrix function
    cmtx = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_pred, labels=[1,0]), 
        index=['true:bad', 'true:good'], 
        columns=['pred:bad','pred:good']
    )
    sensitivty = cmtx['pred:good']['true:good']/float(cmtx['pred:good']['true:good'] + cmtx['pred:bad']['true:bad'])
    precision = cmtx['pred:good']['true:good']/float(cmtx['pred:good']['true:good'] + cmtx['pred:bad']['true:good'])
    display(cmtx)
    display(sensitivty)
    display(precision)
    return (score,sensitivty,precision)

In [ ]:

def track_results_classifier(optimization, accuracy, precision, recall, fits, current_classifier, bestParamsDict,show=True):
    current_classifier['OptApproach'].append(optimization)
    current_classifier['Accuracy'].append(accuracy)
    current_classifier['Fits'].append(fits)
    current_classifier['Precision'].append(precision)
    current_classifier['Recall'].append(recall)
    current_classifier['BestParams'].append(bestParamsDict)
    #{'criterion': 'gini', 'max_features': 1, 'min_samples_leaf': 1, 'n_estimators': 50, 'n_jobs': 2, 'random_state': 1}
    if show:
        df = pd.DataFrame(current_classifier)
        df = df.sort_values('Accuracy', ascending=True)
        display(df)
       # with open('test_02.csv', 'w') as f:
       #   f.write(df)  
        df.to_csv("test_02.csv", encoding='utf-8', index=False)
    return current_classifier    
setup_classifier = {'OptApproach': [],
         'Accuracy':[],
         'Fits':[],
         'Precision':[],
         'Recall':[],
         'BestParams':[],
        } 

defaults_classifier = {
    'n_estimators': [],
         'max_depth': [],
         'min_child_weight': [],
         'learning_rate': [],
         'subsample': [],
         'reg_lambda': [],
         'reg_alpha': [],
}


current_classifier=track_results_classifier('RandomClassifier',prec_score_micro, prec_score_macro, prec_score_wtd, 100, setup_classifier,best_param_dict_rc,show=True)

,OptApproach,Accuracy,Fits,Precision,Recall,BestParams
0,RandomClassifier,0.435811,100,0.182283,0.411639,"{'criterion': 'gini', 'n_estimators': 25, 'ran..."


In [ ]:
#Keep this random seed
#np.random.seed(123)
from sklearn.model_selection import GridSearchCV
# run GridSearchCV with our xgbr_model to find better hyperparameters
# from sklearn.model_selection import GridSearchCV

# define the grid
params_class_01 = {
    #"max_depth": [3],
     #         "max_features": [1, 3, 10],
      #        "min_samples_split": [1.0],
       #       "min_samples_leaf": [1, 3, 10],
              # "bootstrap": [True, False],
              "criterion": ["gini"],
              "n_estimators":[25,50],
              "n_jobs":[2],
              "random_state":[1],
              #"max_depth": [3],
              "max_features": [1, 3, 10],
              #"min_samples_split": [1.0],
              "min_samples_leaf": [1, 3, 10],
}

# setup the grid search
def gridSearch(params_class):
  grid_search_class = GridSearchCV(clf,
                            param_grid=params_class,
                            cv=3,
                            verbose=1,
                            n_jobs=1,
                            return_train_score=True)

  grid_search_class.fit(X_train, y_train)

  y_pred_gs = grid_search_class.predict(X_test)
  #print(grid_search_class.best_params_)
  grid_score = grid_search_class.score(X_test, y_test)
  #print(grid_score)
  #prec_score_micro = precision_score(y_test, y_pred, average='micro')
  prec_score_macro = precision_score(y_test, y_pred_gs, average='macro')
  prec_score_wtd=precision_score(y_test, y_pred_gs, average='weighted')
  return grid_score,prec_score_macro,prec_score_wtd,grid_search_class.best_params_
'''
prec_score_micro_01,prec_score_macro_01,prec_score_wtd_01,best_params_01 = gridSearch(params_class_01)
current_classifier=track_results_classifier('GridSearch_01',prec_score_micro_01, prec_score_macro_01, prec_score_wtd_01, 50, setup_classifier,best_params_01,show=True )
params_class_02 = {
    "n_estimators":[100],
    "criterion":["gini", "entropy", "log_loss"],
    "max_depth": [1,3,5,10],
    "min_samples_split":[2],
    "min_samples_leaf":[1],
    "min_weight_fraction_leaf":[0.0],
    "max_features":["sqrt", "log2", None],
    "max_leaf_nodes":[None],
    "min_impurity_decrease":[0.0],
    "bootstrap":[True],
    "oob_score":[False],
    "n_jobs":[None],
    "random_state":[None],
    "verbose":[0],
    "warm_start":[False],
    "class_weight":["balanced", "balanced_subsample"],
    "ccp_alpha":[0.0],
    "max_samples":[None],
}



prec_score_micro_02,prec_score_macro_02,prec_score_wtd_02,best_params_02 = gridSearch(params_class_02)
current_classifier=track_results_classifier('GridSearch_02',prec_score_micro_02,prec_score_macro_02,prec_score_wtd_02, 50, setup_classifier,best_params_02,show=True )


params_class_03 = {
    "n_estimators":[200],
    "criterion":["gini", "entropy", "log_loss"],
    "max_depth": [1,3,5],
    "min_samples_split":[2,5],
    "min_samples_leaf":[1,2,5],
    "min_weight_fraction_leaf":[1.0,2.0],
    "max_features":["sqrt", "log2", None],
    "max_leaf_nodes":[1,2,5,10,20],
    "min_impurity_decrease":[1.0,2.0],
    "bootstrap":[True],
    "oob_score":[True,False],
    "n_jobs":[1,3,5,10,20,50],
    "random_state":[1,3,5,10,20,50],
    "verbose":[0],
    "warm_start":[False],
    "class_weight":["balanced", "balanced_subsample"],
    "ccp_alpha":[1.0,2.0],
    "max_samples":[1,3,5],
}


#prec_score_micro_03,prec_score_macro_03,prec_score_wtd_03,best_params_03 = gridSearch(params_class_02)
#current_classifier=track_results_classifier('GridSearch_03',prec_score_micro_03,prec_score_macro_03,prec_score_wtd_03, 50, setup_classifier,best_params_03,show=True )

params_class_04 = {
    "n_estimators":[200],
    "criterion":["entropy"],
    "max_depth": [10,15,20],
    "min_samples_split":[7,9,11],
    "min_samples_leaf":[7,9,11],
    "min_weight_fraction_leaf":[1.0,2.0],
    "max_features":["log2"],
    "max_leaf_nodes":[10,20],
    "min_impurity_decrease":[1.0,2.0],
    "bootstrap":[True],
    "oob_score":[True,False],
    "n_jobs":[1,3,5,10,20,50],
    "random_state":[1,3,5],
    "verbose":[0],
    "warm_start":[False],
    "class_weight":["balanced", "balanced_subsample"],
    "ccp_alpha":[1.0,2.0],
    "max_samples":[1,3,5],
}
prec_score_micro_04,prec_score_macro_04,prec_score_wtd_04,best_params_04 = gridSearch(params_class_04)
current_classifier=track_results_classifier('GridSearch_04',prec_score_micro_04,prec_score_macro_04,prec_score_wtd_04, 50, setup_classifier,best_params_04,show=True )
'''


'\nprec_score_micro_01,prec_score_macro_01,prec_score_wtd_01,best_params_01 = gridSearch(params_class_01)\ncurrent_classifier=track_results_classifier(\'GridSearch_01\',prec_score_micro_01, prec_score_macro_01, prec_score_wtd_01, 50, setup_classifier,best_params_01,show=True )\nparams_class_02 = {\n    "n_estimators":[100],\n    "criterion":["gini", "entropy", "log_loss"],\n    "max_depth": [1,3,5,10],\n    "min_samples_split":[2],\n    "min_samples_leaf":[1],\n    "min_weight_fraction_leaf":[0.0],\n    "max_features":["sqrt", "log2", None],\n    "max_leaf_nodes":[None],\n    "min_impurity_decrease":[0.0],\n    "bootstrap":[True],\n    "oob_score":[False],\n    "n_jobs":[None],\n    "random_state":[None],\n    "verbose":[0],\n    "warm_start":[False],\n    "class_weight":["balanced", "balanced_subsample"],\n    "ccp_alpha":[0.0],\n    "max_samples":[None],\n}\n\n\n\nprec_score_micro_02,prec_score_macro_02,prec_score_wtd_02,best_params_02 = gridSearch(params_class_02)\ncurrent_classifie

# New Section

In [ ]:




params_class_03 = {
    "n_estimators":[200],
    "criterion":["gini", "entropy", "log_loss"],
    "max_depth": [1,3,5],
    "min_samples_split":[2,5],
    "min_samples_leaf":[1,2,5],
    "min_weight_fraction_leaf":[1.0,2.0],
    "max_features":["sqrt", "log2", None],
    "max_leaf_nodes":[1,2,5,10,20],
    "min_impurity_decrease":[1.0,2.0],
    "bootstrap":[True],
    "oob_score":[True,False],
    "n_jobs":[1,3,5,10,20,50],
    "random_state":[1,3,5,10,20,50],
    "verbose":[0],
    "warm_start":[False],
    "class_weight":["balanced", "balanced_subsample"],
    "ccp_alpha":[1.0,2.0],
    "max_samples":[1,3,5],
}


#prec_score_micro_03,prec_score_macro_03,prec_score_wtd_03,best_params_03 = gridSearch(params_class_03)
#current_classifier=track_results_classifier('GridSearch_03',prec_score_micro_03,prec_score_macro_03,prec_score_wtd_03, 50, setup_classifier,best_params_03,show=True )